# <span style="color:#ff5f27;"> 🏦 Loan Analysis Training Pipeline</span>

https://www.kaggle.com/code/faressayah/lending-club-loan-defaulters-prediction 


This notebook:

 * selects features from feature groups to create a feature view
 * creates train/test data with the feature view
 * trains a model to predict loan approvals with the training set
 * evaluates the model on the test set
 * uploads the model along with evaluation data to Hopsworks

## <span style="color:#ff5f27;">📝 Imports </span>


In [1]:
#!pip install --quiet hopsworks 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score
)
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store</span>

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/537749


Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🪄 Create the Feature View</span>


In [4]:
# Retrieve feature groups
fg_loans = fs.get_feature_group(
    name="loans", 
    version=1,
)

fg_applicants = fs.get_feature_group(
    name="applicants", 
    version=1,
)

In [5]:
# Select features for training dataset
selected_features = fg_loans.select_except(["id", "issue_d"]).join(\
            fg_applicants.select_except(["earliest_cr_line", "earliest_cr_line_year", "id"]))

# Uncomment this if you would like to view your selected features
# selected_features.show(5)

In [6]:
feature_view = fs.get_or_create_feature_view(
    name="loans_approvals", 
    version=1,
    description="Loan applicant data",
    labels=["loan_status"],
    query=selected_features,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/537749/fs/533572/fv/loans_approvals/version/1


In [7]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=0.2,
)

Error: Reading data from Hopsworks, using Hive           


DatabaseError: Execution failed on sql: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`loan_amnt` `loan_amnt`, `fg1`.`term` `term`, `fg1`.`int_rate` `int_rate`, `fg1`.`installment` `installment`, `fg1`.`sub_grade` `sub_grade`, `fg1`.`loan_status` `loan_status`, `fg1`.`purpose` `purpose`, `fg1`.`zip_code` `zip_code`, `fg1`.`id` `join_pk_id`, `fg1`.`issue_d` `join_evt_issue_d`, `fg0`.`home_ownership` `home_ownership`, `fg0`.`annual_inc` `annual_inc`, `fg0`.`verification_status` `verification_status`, `fg0`.`dti` `dti`, `fg0`.`open_acc` `open_acc`, `fg0`.`pub_rec` `pub_rec`, `fg0`.`revol_bal` `revol_bal`, `fg0`.`revol_util` `revol_util`, `fg0`.`total_acc` `total_acc`, `fg0`.`initial_list_status` `initial_list_status`, `fg0`.`application_type` `application_type`, `fg0`.`mort_acc` `mort_acc`, `fg0`.`pub_rec_bankruptcies` `pub_rec_bankruptcies`, RANK() OVER (PARTITION BY `fg1`.`id`, `fg1`.`issue_d` ORDER BY `fg0`.`earliest_cr_line` DESC) pit_rank_hopsworks
FROM `airquality_don123_featuresto

In [ ]:
X_test.head(3)

In [ ]:
y_train.head(3)

## <span style="color:#ff5f27;">👩🏻‍🔬 Feature Transformation</span>


In [ ]:
# Map features to transformation functions using list comprehensions
# Identify categorical features by checking the data type of each column
categorical_features = [
    col 
    for col 
    in X_train.columns 
    if X_train[col].dtype == object
]

# Identify numeric features by checking the data type of each column
numeric_features = [
    col 
    for col 
    in X_train.columns 
    if X_train[col].dtype != object
]

# Print the identified numeric and categorical features
print("⛳️ Numeric Features:", numeric_features)
print("⛳️ Categorical Features:", categorical_features)

In [ ]:
# Define a numeric transformer pipeline
numeric_transformer = Pipeline(
    steps=[
        # Impute missing values with the median and scale the numeric features
        ("imputer", SimpleImputer(strategy="median")), 
        ("scaler", StandardScaler()),
    ]
)

# Define a categorical transformer pipeline
categorical_transformer = Pipeline(
    steps=[
        # Encode categorical features using one-hot encoding and select top features using chi-squared test
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)

# Use ColumnTransformer to apply transformers to different subsets of columns
# Here, numeric features are processed by the numeric_transformer,
# and categorical features are processed by the categorical_transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),  # Apply numeric transformer to numeric features
        ("cat", categorical_transformer, categorical_features),  # Apply categorical transformer to categorical features
    ]
)

In [ ]:
# Map the 'loan_status' column in y_train to binary labels
y_train['loan_status'] = y_train.loan_status.map({'Fully Paid': 1, 'Charged Off': 0})

# Map the 'loan_status' column in y_test to binary labels
y_test['loan_status'] = y_test.loan_status.map({'Fully Paid': 1, 'Charged Off': 0})

In [ ]:
y_train.head()

## <span style="color:#ff5f27;">🤖 Models Building</span>


In [ ]:
def print_score(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

In [ ]:
# Create a pipeline with preprocessing and a logistic regression classifier
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),  # Apply the specified preprocessor (ColumnTransformer)
        ("classifier", LogisticRegression()),  # Use Logistic Regression as the classifier
    ]
)

# Fit the pipeline on the training data
clf.fit(X_train, y_train['loan_status'].ravel())

In [ ]:
# Predictions on the training set
y_train_pred = clf.predict(X_train)

# Predictions on the test set
y_test_pred = clf.predict(X_test)

# Print performance scores for the training set
print_score(y_train, y_train_pred, train=True)

# Print performance scores for the test set
print_score(y_test, y_test_pred, train=False)

In [ ]:
# Create a Confusion Matrix display from the classifier's predictions on the test set
disp = ConfusionMatrixDisplay.from_estimator(
    clf,  # Classifier pipeline
    X_test,  # Test set features
    y_test,  # True labels for the test set
    cmap='Blues',  # Color map for the display
    values_format='d',  # Format for the values in the display
    display_labels=['Default', 'Fully-Paid'],  # Labels for the display
)

# Create a ROC Curve display from the classifier's predictions on the test set
display = RocCurveDisplay.from_estimator(clf, X_test, y_test)

# Plot the ROC Curve
display.plot()

## <span style="color:#ff5f27;">🗄️ Register the Model with Model Registry</span>


In [ ]:
# Get the model registry
mr = project.get_model_registry()

In [ ]:
# Create a directory "features" within the "lending_model" directory, if it doesn't exist
os.makedirs("lending_model/features", exist_ok=True)

# Save the ROC Curve plot as 'roc_curve.png' in the "lending_model" directory
plt.savefig('lending_model/roc_curve.png')

# Save the trained classifier pipeline as 'lending_model.pkl' in the "lending_model" directory
joblib.dump(clf, 'lending_model/lending_model.pkl')

# Calculate and print the ROC AUC score on the test set
accuracy = roc_auc_score(y_test, clf.predict(X_test))
print("⛳️ ROC AUC Score on Test Set:", accuracy)

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/3.0/user_guides/mlops/registry/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define an input schema
input_schema = Schema(X_test)

# Define an output schema
output_schema = Schema(y_test)

# Define a model schema
model_schema = ModelSchema(
    input_schema=input_schema,    # Specify the input schema
    output_schema=output_schema,  # Specify the output schema
)

## <span style="color:#ff5f27;">📝 Register model</span>

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
# Create a sklearn model in the Model Registry
fraud_model = mr.sklearn.create_model(
    "lending_model",
    metrics={'accuracy': accuracy},            # Specify metrics for the model
    input_example=X_test.sample().to_numpy(),  # Provide an input example for the model
    model_schema=model_schema,                 # Specify the model schema
)

# Save the created model in the "lending_model" directory
fraud_model.save('lending_model')

---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Batch Inference</span>

In the following notebook you will use your model for batch inference.
